In [1]:
import os
from docx import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter
from chromadb import Client
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
from dotenv import load_dotenv, find_dotenv
import openai
from sentence_transformers import CrossEncoder
import numpy as np
from openai import OpenAI

# Load environment variables
_ = load_dotenv(find_dotenv())

client = OpenAI(api_key = os.getenv('OPENAI_API_KEY'))

# Initialize ChromaDB client
chroma_client = Client()


/home/mahbubah/Desktop/week-11/.venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
def preprocess_document(docx_path):
    doc = Document(docx_path)
    docx_texts = [paragraph.text.strip() for paragraph in doc.paragraphs if paragraph.text.strip()]

    character_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n", ". ", " ", ""], chunk_size=1000, chunk_overlap=0)
    character_split_texts = character_splitter.split_text('\n\n'.join(docx_texts))

    token_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=0, tokens_per_chunk=256)
    token_split_texts = []
    for text in character_split_texts:
        token_split_texts += token_splitter.split_text(text)

    return token_split_texts

# Example document path
docx_path = "../Evaluation Sets/Robinson Advisory.docx"
documents = preprocess_document(docx_path)

# Embedding function
embedding_function = SentenceTransformerEmbeddingFunction()

# Create ChromaDB collection and add documents
chroma_collection = chroma_client.create_collection("Contract", embedding_function=embedding_function)
ids = [str(i) for i in range(len(documents))]
chroma_collection.add(ids=ids, documents=documents)


In [3]:
def query_documents(query, collection):
    results = collection.query(query_texts=[query], n_results=5)
    retrieved_documents = results['documents'][0]
    return retrieved_documents

In [4]:

def augment_multiple_query(query, model="gpt-3.5-turbo"):
    messages = [
        {
            "role": "system",
            "content": "You are a helpful expert contract advisor assistant. Your users are asking questions about information contained in the contract."
                       "Suggest up to five additional related questions to help them find the information they need, for the provided question. "
                       "Suggest only short questions without compound sentences. Suggest a variety of questions that cover different aspects of the topic."
                       "Make sure they are complete questions, and that they are related to the original question."
                       "Output one question per line. Do not number the questions."
        },
        {"role": "user", "content": query}
    ]

    response = client.chat.completions.create(
        model=model,
        messages=messages,
    )
    content = response.choices[0].message.content
    content = content.split("\n")
    return content


In [5]:
def rag_with_query_expansion(original_query, model="gpt-3.5-turbo"):
    # Expand the original query
    expanded_queries = augment_multiple_query(original_query, model=model)[:5]  # Limit to top 5 expanded queries

    # Include the original query in the list of queries
    queries = [original_query] + expanded_queries

    # Query ChromaDB for relevant documents
    results = chroma_collection.query(query_texts=queries, n_results=5, include=['documents'])
    retrieved_documents = results['documents']

    # Deduplicate the retrieved documents
    unique_documents = set()
    for documents in retrieved_documents:
        for document in documents:
            unique_documents.add(document)

    # Convert back to list for indexing
    unique_documents = list(unique_documents)

    # Rank documents based on relevance to each query
    cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
    pairs = [[query, doc] for doc in unique_documents for query in queries]
    scores = cross_encoder.predict(pairs)

    # Sort documents based on average score across all queries
    avg_scores = np.mean(np.array(scores).reshape(len(queries), -1), axis=0)
    ranked_indices = np.argsort(avg_scores)[::-1]
    ranked_documents = [unique_documents[i] for i in ranked_indices]

    # Generate answer using RAG with ranked documents
    information = "\n\n".join(ranked_documents[:5])  # Use top 5 ranked documents
    print(information)
    
    messages = [
        {
            "role": "system",
            "content": "You are a helpful expert legal contract advisor assistant. Your users are asking questions about information contained in the contract."
                       "You will be shown the user's question, and the relevant information from the contract. Answer the user's question using only this information."
        },
        {"role": "user", "content": f"Question: {original_query}. \n Information: {information}"},
        
    ]
    
    response = client.chat.completions.create(
        model=model,
        messages=messages,
    )
    content = response.choices[0].message.content
    return content


In [6]:
# Main script
if __name__ == "__main__":
    #query = "What is the purpose of the escrow? "
    
    original_query = "In which street does the Advisor live?"
    
    # Generate answer with query expansion and RAG
    output = rag_with_query_expansion(original_query)
   
    
    # Print or use the answer as needed
    print("Generated Answer:")
    print(output)


ip : any work product, upon creation, shall be fully and exclusively owned by the company. the advisor, immediately upon company ’ s request, shall sign any document and / or perform any action needed to formalize such ownership. the advisor shall not obtain any rights in the work product, including moral rights and / or rights for royalties or other consideration under any applicable law ( including section 134 of the israeli patent law – 1967 if applicable ), and shall not be entitled to any compensation with respect to the services, which was not specifically agreed, in writing, between the advisor and the company. non - compete : during the term of engagement with the company and for a period of 12 months thereafter, advisor shall not be involved, as an employee, owner, contractor or otherwise, in any business, which competes with the company ’ s business, and shall not solicit and / or hire any employee and / or service provider of the company, without the prior written consent of

In [7]:
def retrieve_documents(query):
    # Perform your retrieval logic here
    results = chroma_collection.query(query_texts=[query], n_results=5)
    retrieved_documents = results['documents'][0]  # Assuming the structure of returned documents
    return retrieved_documents

In [8]:
from datasets import Dataset

# Define your questions, answers, and ground truths
questions = [
    "Is there a non-compete obligation to the Advisor?",
    "Can the Advisor charge for meal time?",
    "In which street does the Advisor live?",
    "Is the Advisor entitled to social benefits?",
]

ground_truths = [
    "Yes. During the term of engagement with the Company and for a period of 12 months thereafter.",
    "No. See Section 6.1, Billable Hour doesn’t include meals or travel time.",
    "1 Rabin st, Tel Aviv, Israel",
    "No. According to section 8 of the Agreement, the Advisor is an independent consultant and shall not be entitled to any overtime pay, insurance, paid vacation, severance payments or similar fringe or employment benefits from the Company.",
]

answers = []
contexts = []


for query in questions:
    answers.append(rag_with_query_expansion(query))
    relevant_documents = retrieve_documents(query)
    #contexts.append([doc.page_content for doc in relevant_documents])
    contexts.append(relevant_documents)  # Already contains text content
    #contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])

# Create the dataset dictionary
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truth": ground_truths  
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)



personnel : the advisor may provide the services only directly or through employees, contractors or agents ( “ personnel ” ), which were specifically approved by the company, and have committed in writing to confidentiality and ownership rights towards the advisor and / or the company, in a form restrictive enough to allow advisor to comply and enforce the commitments under this undertaking ( “ authorized personnel ” ). without limiting the foregoing, the advisor is liable hereunder for any breach of this undertaking by any advisor personnel.

. advisor further consents that the company may offset any amounts due to him under this section from any amounts payable to advisor under this agreement. advisor shall indemnify the company for any loss or expenses incurred by the company if it were determined that an alleged employer / employee relationship existed between the advisor and the company.

advisor shall provide to the company, as an independent contractor, software development serv

In [9]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

result = evaluate(
    dataset=dataset,
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
    raise_exceptions=False, 
    is_async=False
)
df = result.to_pandas()

# Print the dataframe to inspect the results
print(df)


Evaluating: 100%|██████████| 16/16 [00:15<00:00,  1.06it/s]


                                            question  \
0  Is there a non-compete obligation to the Advisor?   
1              Can the Advisor charge for meal time?   
2             In which street does the Advisor live?   
3        Is the Advisor entitled to social benefits?   

                                              answer  \
0  Yes, there is a non-compete obligation for the...   
1  The advisor can only charge for billable hours...   
2  The Advisor lives at 1 Rabin St, Tel Aviv, Isr...   
3  No, the Advisor shall not receive nor be entit...   

                                            contexts  \
0  [advisor shall be solely responsible for any i...   
1  [as full and sole consideration for the servic...   
2  [advisor shall provide to the company, as an i...   
3  [relationship of the parties ; indemnification...   

                                        ground_truth  context_precision  \
0  Yes. During the term of engagement with the Co...             1.0000   
1  No. 

In [10]:
#metrices = df['context_precision','context_recall','faithfulness','answer_relevancy']
#metrices

In [11]:
print(result)

{'context_precision': 0.7719, 'context_recall': 0.7500, 'faithfulness': 1.0000, 'answer_relevancy': 0.9359}


In [12]:
import plotly.graph_objects as go


data = {
    'context_precision': result['context_precision'],
    'faithfulness': result['faithfulness'],
    'answer_relevancy': result['answer_relevancy'],
    'context_recall': result['context_recall'],
    }

fig = go.Figure()

fig.add_trace(go.Scatterpolar(
    r=list(data.values()),
    theta=list(data.keys()),
    fill='toself',
    name='Ensemble RAG'
))

fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 1]
        )),
    showlegend=True,
    title='Retrieval Augmented Generation - Evaluation',
    width=800,
)

fig.show()